![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Use watsonx, and LangChain to make a series of calls to a language model

#### Disclaimers

- Use only Projects and Spaces that are available in watsonx context.


## Notebook content

This notebook contains the steps and code to demonstrate Simple Sequential Chain using langchain integration with watsonx models.

Some familiarity with Python is helpful. This notebook uses Python 3.10.


## Learning goal

The goal of this notebook is to demonstrate how to chain `google/flan-ul2` and `google/flan-t5-xxl` models to generate a sequence of creating a random question on a given topic and an answer to that question and also to make the user friends with LangChain framework, using simple chain (LLMChain) and the extended chain (SimpleSequentialChain) with the WatsonxLLM.


## Contents

This notebook contains the following parts:

- [Setup](#setup)
- [Foundation Models on watsonx](#models)
- [LangChain integration](#watsonxllm)
- [Simple Sequential Chain experiment](#experiment)
- [Summary](#summary)

<a id="setup"></a>
## Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Install and import the `datasets` and dependecies

In [2]:
!pip install "ibm-watson-machine-learning>=1.0.327" | tail -n 1
!pip install "pydantic>=1.10.0" | tail -n 1
!pip install "langchain==0.0.340" | tail -n 1

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a v

### Defining the WML credentials
This cell defines the WML credentials required to work with watsonx Foundation Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [23]:
import getpass

credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "API_KEY"
}
project_id = "PROJECT_ID"

<a id="models"></a>
## Foundation Models on `watsonx.ai`

#### List available models

All avaliable models are presented under `ModelTypes` class.

In [4]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q']


You need to specify `model_id`'s that will be used for inferencing:

In [5]:
model_id_1 = ModelTypes.FLAN_UL2
model_id_2 = ModelTypes.FLAN_T5_XXL

### Defining the model parameters

You might need to adjust model `parameters` for different models or tasks, to do so please refer to documentation under `GenTextParamsMetaNames` class.

**Action:** If any complications please refer to the <a href="https://ibm.github.io/watson-machine-learning-sdk/" target="_blank" rel="noopener no referrer">documentation</a>.

In [6]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain.chains.llm import LLMChain


parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE,
    GenParams.MAX_NEW_TOKENS: 100,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

### Initialize the model
Initialize the `Model` class with previous set params.

In [7]:
from ibm_watson_machine_learning.foundation_models import Model

flan_ul2_model = Model(
    model_id=model_id_1, 
    params=parameters, 
    credentials=credentials,
    project_id=project_id)

flan_t5_model = Model(
    model_id=model_id_2,
    credentials=credentials,
    project_id=project_id)

<a id="watsonxllm"></a>
## LangChain integration

`WatsonxLLM` is a wrapper around watsonx.ai models that provide chain integration around the models.

**Action:** For more details about `CustomLLM` check the <a href="https://python.langchain.com/docs/modules/model_io/models/llms/custom_llm" target="_blank" rel="noopener no referrer">LangChain documentation</a>


### Initialize the `WatsonxLLM` class.

In [8]:
from ibm_watson_machine_learning.foundation_models.extensions.langchain import WatsonxLLM

flan_ul2_llm = WatsonxLLM(model=flan_ul2_model)
flan_t5_llm = WatsonxLLM(model=flan_t5_model)

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.

In [9]:
flan_ul2_model.to_langchain()

WatsonxLLM(model=<ibm_watson_machine_learning.foundation_models.model.Model object at 0x28fe939d0>)

You can print all set data about the WatsonxLLM object using the `dict()` method.

In [10]:
flan_ul2_llm.dict()

{'model_id': 'google/flan-ul2',
 'params': {'decoding_method': <DecodingMethods.SAMPLE: 'sample'>,
  'max_new_tokens': 100,
  'min_new_tokens': 1,
  'temperature': 0.5,
  'top_k': 50,
  'top_p': 1},
 'project_id': '838be5fc-4383-452e-937f-4d33516b6232',
 'space_id': None,
 '_type': 'IBM watsonx.ai'}

In [11]:
##predict with the model
text = "Where is the capital of South Korea"
flan_ul2_llm(text)

'seoul'

## Prompt Templates & Chains

In the previous example, the user input is sent directly to the LLM. However, when using an LLM in an application, you will usually need to reuse the same prompt across multiple scenarios

- Accepting user input and contruct a prompt
- Generating mutiple prompts from an collection of data points in a dataset 

In [12]:
# Define the prompt templates
prompt = PromptTemplate(
  input_variables=["country"],
  template= "where is the capital of {country}?",
)

# Chaining 
chain = LLMChain(llm=flan_ul2_llm, prompt=prompt)

# Getting predictions
countries = ["USA", "England", "Japan", "France"]
for country in countries:
    response = chain.run(country)
    print(prompt.format(country=country) + " = " + response)

where is the capital of USA? = washington dc
where is the capital of England? = london
where is the capital of Japan? = tokyo
where is the capital of France? = paris


<a id="experiment"></a>
## Simple Sequential Chain experiment
The simplest type of sequential chain is called a `SimpleSequentialChain`, in which each step has a single input and output and the output of one step serves as the input for the following step.

The experiment will consist in generating a random question about any topic and answer the following question.

An object called `PromptTemplate` assists in generating prompts using a combination of user input, additional non-static data, and a fixed template string.

In our case we would like to create two `PromptTemplate` objects which will be responsible for creating a random question and answering it.

In [13]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

We would like to add functionality around language models using `LLMChain` chain.

`prompt_to_flan_ul2` chain formats the prompt template whose task is to generate random question, passes the formatted string to LLM and returns the LLM output.

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.


In [14]:
from langchain.chains import LLMChain

prompt_to_flan_ul2 = LLMChain(llm=flan_ul2_model.to_langchain(), prompt=prompt_1)

`flan_to_t5` chain formats the prompt template whose task is to answer the question we got from `prompt_to_flan_ul2` chain, passes the formatted string to LLM and returns the LLM output.

In [15]:
flan_to_t5 = LLMChain(llm=flan_t5_model.to_langchain(), prompt=prompt_2)

This is the overall chain where we run `prompt_to_flan_ul2` and `flan_to_t5` chains in sequence.

In [16]:
from langchain.chains import SimpleSequentialChain

qa = SimpleSequentialChain(chains=[prompt_to_flan_ul2, flan_to_t5], verbose=True)

Generate random question and answer to topic.

In [17]:
qa.run('cars')



> Entering new SimpleSequentialChain chain...


What was the name of the first car to use a rotary engine?
mazda rx-7

> Finished chain.


'mazda rx-7'

## Using Langchain for Summarization

### Loading Web Document Using Langchain Loaders

In [18]:
from langchain.document_loaders import WebBaseLoader
from langchain.chains.combine_documents.stuff import StuffDocumentsChain


# Initialize llm and document loader:
print("Loading web document...")
# Try out some other documents as well
loader = WebBaseLoader("https://www.ibm.com/blog/reducing-defects-and-downtime-with-ai-enabled-automated-inspections/")
doc = loader.load()
print("Done.")


Loading web document...
Done.


### Initializing Model

In [19]:

# You might need to tweak some of the runtime parameters to optimize the results.
print("Initializing mixtral-8x7b model...")
params = {
    GenParams.DECODING_METHOD: "sample",
    GenParams.TEMPERATURE: 0.15,
    GenParams.TOP_P: 1,
    GenParams.TOP_K: 20,
    GenParams.REPETITION_PENALTY: 1.0,
    GenParams.MIN_NEW_TOKENS: 20,
    GenParams.MAX_NEW_TOKENS: 205,
    GenParams.STOP_SEQUENCES: ["\n"]
}

mixtral_model = Model(
    model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q",
    params=params,
    credentials=credentials,
    project_id=project_id
).to_langchain()



Initializing mixtral-8x7b model...


### Defining Prompt

In [20]:
# Define prompt
prompt_template = """Write a concise summary of the following article in one paragraph:
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)


### Running Summarization Chain

In [21]:

# Define LLM chain
print("Initializing chain...")
llm_chain = LLMChain(llm=mixtral_model, prompt=prompt)

# Define StuffDocumentsChain
print("Stuff chain with documents...")
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, document_variable_name="text"
)

print("Running summarization on stuffed document chain...\n")
res = stuff_chain.run(doc)

print(res)

print("\nDone.")

Initializing chain...
Stuff chain with documents...
Running summarization on stuffed document chain...



A multinational automobile manufacturer partnered with IBM to implement AI-enabled automated inspections using IBM's Inspection Suite, including fixed-mounted, handheld, and wearable inspections. The solution, which is based on standard iPhones and uses readily available hardware, improved the client's quality inspection process without requiring coding. The system learned quickly from images of acceptable and defective work products, enabling the solution to be up and running within a matter of weeks. The implementation resulted in a significant reduction in defects and over 30 million inspections, leading to the client awarding IBM their annual IT Innovation award. The collaboration also enabled continuous process improvement, lower repair and warranty costs, and improved customer satisfaction.

Done.


<a id="summary"></a>
## Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use Simple Squential Chain using custom llm WastonxLLM.
 
Check out our _<a href="https://ibm.github.io/watson-machine-learning-sdk/samples.html" target="_blank" rel="noopener no referrer">Online Documentation</a>_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors: 
 **Mateusz Szewczyk**, Software Engineer at Watson Machine Learning.

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.